In [1]:
import csv
import pandas as pd
import numpy as np

import folium
import folium.plugins as plugins
import ipywidgets as widgets
from IPython.display import HTML
from IPython.display import display

#parse and load csv file into a Pandas DataFrame
csvfile = 'https://raw.githubusercontent.com/rkalyanapurdue/smolensk/master/Division.csv'
df = pd.read_csv(csvfile,parse_dates=['MAP_DATE'])

#group the data by Army group and num
grouped_div = df.groupby(['Army_Group','Num_Name'])

#create a dictionary of the army groups and nums to use in a selection widget
army_groups = dict()
for group_keys in grouped_div.groups.keys():
    army_group = group_keys[0]
    div_num = group_keys[1]
    if isinstance(div_num,str):
        if army_group in army_groups:
            army_groups[army_group].append(div_num)
        else:
            army_groups[army_group] = ['-',div_num]

#output widget for the map
out = widgets.Output()

#selection widgets for army group and num
group_sel = widgets.Dropdown(options=['-']+list(army_groups.keys()))
div_sel = widgets.Dropdown(options=['-'])

#event handler
def on_group_sel(change):
    if change['new'] in army_groups:
        div_sel.options = army_groups[change['new']]
        plot_div_movement(change['new'],army_groups[change['new']][0])
    if change['new'] == '-':
        div_sel.options = ['-']
        plot_div_movement(change['new'],'-')

group_sel.observe(on_group_sel,'value')

def on_div_sel(change):
    if change['new'] is not None:
        plot_div_movement(group_sel.value,change['new'])
    
div_sel.observe(on_div_sel,'value')

#plots a trail of locations with lines connecting them
def add_trail_to_map(dest_map,locations):
    #only if there is movement
    if len(locations) > 1:
        #first the position markers
        folium.CircleMarker(locations[0],radius=4,color='green').add_to(dest_map)
        folium.CircleMarker(locations[len(locations)-1],radius=4,color='red').add_to(dest_map)
        #next the vectors
        #for i in range(0,len(locations)-1):
        #    vect_pair = [locations[i],locations[i+1]]
        #    vect_line = folium.PolyLine(locations=vect_pair,weight=1,color='black')
        #    dest_map.add_child(vect_line)
        vect_pair = [locations[0],locations[len(locations)-1]]
        vect_line = folium.PolyLine(locations=vect_pair,weight=1,color='black')
        dest_map.add_child(vect_line)
    
#main function to plot the movement of a selected group and num
def plot_div_movement(army_group,num_name):
    division_map = folium.Map([54.78, 32.04],zoom_start=6)
    if army_group == '-':
        #plot all division vectors
        for group_key in grouped_div.groups.keys():
            #weed out groups that don't have a valid num
            if isinstance(group_key[1],str):
                div_df = grouped_div.get_group(group_key)[['POINT_X','POINT_Y']]
                coordinates = [(row.POINT_Y,row.POINT_X) for row in div_df.itertuples()]
                add_trail_to_map(division_map,coordinates)
    else:
        #specific army group and num
        #default is to display all division nums for this group
        if num_name == '-':
            for group_key in grouped_div.groups.keys():
                #weed out groups that don't have a valid num and groups that are not the selected one
                if isinstance(group_key[1],str) and group_key[0] == army_group:
                    div_df = grouped_div.get_group(group_key)[['POINT_X','POINT_Y']]
                    coordinates = [(row.POINT_Y,row.POINT_X) for row in div_df.itertuples()]
                    add_trail_to_map(division_map,coordinates)
        else:
            group_key = (army_group,num_name)
            div_df = grouped_div.get_group(group_key)[['POINT_X','POINT_Y']]
            coordinates = [(row.POINT_Y,row.POINT_X) for row in div_df.itertuples()]
            add_trail_to_map(division_map,coordinates)
            
    out.clear_output()
    with out:
        iframe = division_map._repr_html_()
        display(HTML(iframe))
    return

#output UI
sel_ui = widgets.HBox([group_sel,div_sel])
res = widgets.VBox([sel_ui,out])
plot_div_movement('-','-')
res